In [ ]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional

# Define the schema for the extracted data
class DimeUSStock(BaseModel):
    model_config = ConfigDict(extra="ignore")

    Position: Optional[str] = Field(
        description="Buy or Sell", examples=["Buy", "Sell"], min_length=3, max_length=4
    )
    Ticker: Optional[str] = Field(
        description="Stock Ticker"
    )
    Market: Optional[str] = Field(
        description="Market"
    )
    Executed_Price: Optional[str] = Field(
        alias="Executed Price", description="Executed price in THB or USD"
    )
    Shares: Optional[str] = Field(
        alias="Shares", description="Number of shares bought or sold"
    )
    Stock_Amount: Optional[str] = Field(
        alias="Stock Amount", description="Amount of stock in THB or USD"
    )
    Commission_Fee: Optional[str] = Field(
        alias="Commission Fee", description="Commission Fee in THB or USD"
    )
    VAT: Optional[str] = Field(
        alias="VAT 7%", description="VAT 7% in THB or USD"
    )
    Submission_Date: Optional[str] = Field(
        alias="Submission Date"
    )
    Completion_Date: Optional[str] = Field(
        alias="Completion Date"
    )

# Initialize LLaVA model (replace this with your LLaVA model initialization)
# Note: This assumes you have access to a LangChain-compatible LLaVA model API.
model = Ollama(model="llava")

# Define the folder containing the images
folder_path = "/Users/pupipatsingkhorn/Developer/repositories/Investment-Portfolio/data/private/receipt/dime/us_stock_test"  # Update this path

# Initialize the parser and prompt template
parser = JsonOutputParser(pydantic_object=DimeUSStock)

prompt_template = PromptTemplate(
    template="Extract information and answer in JSON format like this:\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Process each image in the folder
extracted_data = []

for filename in tqdm(sorted(os.listdir(folder_path))):
    if any(ext in filename.upper() for ext in [".PNG", ".JPEG", ".JPG"]):
        file_path = os.path.join(folder_path, filename)

        # Load image
        image = Image.open(file_path)

        # Prepare query (LLaVA can take the image directly)
        query = f"""
        Perform OCR on this image and extract useful information.

        Image: {image}
        """

        # Run the chain
        chain = prompt_template | model | parser
        result = chain.invoke({"query": query})

        extracted_data.append(result)

# Convert the results to a DataFrame
df = pd.DataFrame(extracted_data)
df